# Probabilistic Neural Network : Contoh Kasus
[Iqbal Basyar](https://github.com/underground-11), 2018

*Notebook ini digunakan untuk referensi tugas 1.3 Machine Learning : PNN* <br>
Deksripsi tugas : [link](https://github.com/underground-11/Semester_6/blob/master/Asdos%20Machine%20Learning/Tugas%203%20Machine%20Learning/Tugas1.3.pdf) <br>
Data Train :  [link](https://github.com/underground-11/Semester_6/blob/master/Asdos%20Machine%20Learning/Tugas%203%20Machine%20Learning/data_train_PNN.txt) <br>
Data Test (no label): [link](https://github.com/underground-11/Semester_6/blob/master/Asdos%20Machine%20Learning/Tugas%203%20Machine%20Learning/data_test_PNN.txt) <br>

*Catatan : Install Plotly di mesin anda untuk menggunakan plot dalam notebook ini*



In [126]:
# Kebutuhan library
import numpy as np
import pandas as pd
import plotly
import plotly.graph_objs as go
import matplotlib
import matplotlib.pyplot as plt
from numpy import linalg as la
from plotly.offline import *
from plotly.graph_objs import *
from plotly import __version__
# print ('Plotly Version : ',__version__) # requires version >= 1.9.0
init_notebook_mode(connected=True) #biar bisa inline plot plotly

## 1. Load data to Dataframe

In [127]:
dataTrain = pd.read_csv('data_train_PNN.txt', delimiter = "\t").round(9)
dataTest = pd.read_csv('data_test_PNN.txt', delimiter = "\t").round(9)

Pisahkan dataTrain untuk mempermudah plot

In [140]:
kelas1 = dataTrain.loc[dataTrain['label'] == 0]
kelas2 = dataTrain.loc[dataTrain['label'] == 1]
kelas3 = dataTrain.loc[dataTrain['label'] == 2]

Namun, data diatas masih memiliki indeks yang lama (sesuai index di dataframe sebelumnya). Untuk menghindari kesalahan saat melakukan oeprasi yang berkaitan dengan indeks data, maka data-data baru tersebut harus direset indeksnya. Gunakan fungsi reset_indeks dari pandas sebagai berikut :

In [149]:
kelas1 = kelas1.reset_index(drop = True)
kelas2 = kelas2.reset_index(drop = True)
kelas3 = kelas3.reset_index(drop = True)

## 2. Plot Data Train

In [150]:
trace1 = go.Scatter3d(
    name = 'kelas 0',
    x=kelas1['att1'],
    y=kelas1['att2'],
    z=kelas1['att3'],
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 0, 0.14)',
            width=0.2
        ),
        opacity=0.9
    )
)

trace2 = go.Scatter3d(
    name = 'kelas 1',
    x=kelas2['att1'],
    y=kelas2['att2'],
    z=kelas2['att3'],
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.2
        ),
        opacity=0.9
    )
)

trace3 = go.Scatter3d(
    name = 'Kelas 2',
    x=kelas3['att1'],
    y=kelas3['att2'],
    z=kelas3['att3'],
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.2
        ),
        opacity=0.9
    )
)

data = [trace1, trace2, trace3]

this_layout = go.Layout(
    title = 'Data Train',
#     height = '800' ,
    scene = dict(
        xaxis = dict(
            title='Att 1'),
        yaxis = dict(
            title='Att 2'),
        zaxis = dict(
            title='Att 3'),),
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=25
    )
)

fig = go.Figure(data=data, layout=this_layout)
iplot(fig, filename='Data Train Scatter')

## 3 Fungsi-Fungsi 
Fungsi fungsi yang digunakan dalam PNN antara lain : 
### Fungsi PDF Gaussian 
Fungsi pdf gaussian digunakan untuk menentukan besar peluang sebuah data terhadap kelas data lain di data training.


Fungsi pdf untuk univariate :
$$ \large f_1(x; \sigma, data) = \frac{1}{n\sigma} \sum_{k=1}^{n} e^{-\frac{(x - data_k)^2}{\sigma^2}} $$

fungsi pdf untuk multivariate :
$$ \large f_2(\textbf{x}; \sigma, \textbf{data}) = 
\frac{1}{(2\pi)^\frac{p}{2}  \sigma^p n} \sum_{k=1}^{n} e^ {- \large \frac{||  \textbf{x} - \textbf{data}_k  ||^2}{\sigma^2}} $$


Keterangan : <br>
    $w$: nilai yang diinginkan (data) atau pusat puncak<br>
    $\sigma$: simpangan baku (disebut _Gaussian RMS width_), yang merepresentasikan lebar kurva yang dihasilkan.

Dalam python fungsi ini dapat ditulis dengan:

In [151]:
def f1(x, sigma, dataTrain):
    p = dataTrain.shape[1]  #Mendapatkan banyaknya variabel input data (dimensi data) 
    n = dataTrain.shape[0]  #Mendapatkan banyaknya indeks (row) pada tipe data DataFrame. 
    
    total = 0
    for i in dataTrain.index :
        y = x.sub(dataTrain.index[i],axis=0) ** 2
        y = np.exp(-y / sigma**2)
        total += y 
        
    return (total / (n * sigma))

def f2(x, sigma, dataTrain):
    p = dataTrain.shape[1]  #Mendapatkan banyaknya variabel input data 
    n = dataTrain.shape[0]  #Mendapatkan banyaknya indeks (row) pada tipe data DataFrame. 
    
    total = 0
    for i in dataTrain.index :
        y = la.norm((x.sub(dataTrain.index[i],axis=0)).values) ** 2 #pengurangan data
        # .values akan mengubah dataframe to np array sehingga bisa di norm kan
        y = np.exp(-y / sigma**2)
        total += y 
        
    return (total /(2* 3.14)**(p/2) * sigma**p *n)

### Fungsi untuk mencari sigma ($\sigma$)
Fungsi ini digunakan untuk mendapatkan parameter $\sigma$.<br>
Parameter sigma untuk setiap kelas dapat berbeda-beda. Oleh karena itu, pengaplikasian fungsi ini dikhususkan untuk mendapatkan nilai sigma dari dataset uniform (memiliki label yang sama). Atau, dalam kasus diatas setiap kelas1, kelas2, dan kelas3 akan memiliki nilai sigmanya sendiri


$$G(\textbf{data}) = \large g  \frac{\sum_{k=1}^{n}  d_k}{n}   $$

*Keterangan* : <br>
    g : sebuah konstanta(nantinya akan diobservasi) <br>
    $d_k$ : jarak terpendek data ke-k dengan seluruh data lainnya
    
Dalam python, fungsi tersebut dapat ditulis : 

In [152]:
def G(g, data):
    idx = data.index
    d = []
    for i in idx:
        currentData  = data.loc[i]
        neighborData = data.drop(data.index[i]) #menghapus data pada indeks ke i
        neighborData = neighborData.sub(currentData,axis=1).values #Distance matrix
        norms = [la.norm(j) for j in neighborData ]
        d.append(min(norms)) #menambahkan jarak miniimum data ke i terhadap data lainnya
    return (g * np.mean(d))

Sebagai contoh, jika kita gunakan g = 2, maka $\sigma$ setiap kelas adalah sebagai berikut :

In [155]:
g = 2
print ('Kelas 1: ', G(g, kelas1))
print ('Kelas 2: ', G(g, kelas2))
print ('Kelas 3: ', G(g, kelas3))

Kelas 1:  1.20449389067
Kelas 2:  1.2947942001
Kelas 3:  1.55587034727


# B Pengujian
## B1.a Load data latih dan data uji
Load data telah dilakukan sebelumnya

Berikut spesifikasi data train

In [153]:
# dataTrain = pd.read_csv('data_train_PNN.txt', delimiter = "\t")
# dataTest = pd.read_csv('data_test_PNN.txt', delimiter = "\t")
trace = go.Table(
    header=dict(values=dataTrain.columns,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[dataTrain.att1, dataTrain.att2, dataTrain.att3, dataTrain.label],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))


data = [trace] 
iplot(data, filename = 'Data Train')

Dan berikut spesifikasi data tes

In [154]:
trace = go.Table(
    header=dict(values=dataTest.columns,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[dataTest.att1, dataTest.att2, dataTest.att3],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))


data = [trace] 
iplot(data, filename = 'Data Test')

## B.1.b Klasifikasi
Proses klasifikasi setiap data adalah dengan membandingkan nilai fungsi pdf data tersebut dari masing-masing kelas
. <br>
Hal yang harus diperhatikan adalah parameter g yang akan menentukan nilai sigma($\sigma$) setiap kelas.

Berikut contoh pengaplikasiannya :
### 1. Ubah data kedalam bentuk dataframe

In [159]:
newData = [{'att1' : 1, 'att2': 2, 'att3':3}]
newData = pd.DataFrame(newData)
print (newData)

   att1  att2  att3
0     1     2     3


### 2. Siapkan nilai $\sigma$ untuk masing masing kelas
Dengan asumsi nilai g = 1

In [161]:
g = 1
sigma1 = G(g, kelas1)
sigma2 = G(g, kelas2)
sigma3 = G(g, kelas3)
print (sigma1, sigma2, sigma3)

0.602246945335 0.64739710005 0.777935173637


### 3. Hitung nilai pdf data baru terhadap masing-masing kelas
Karena kasusnya multivariate, maka gunakan f2

In [170]:
nilai_pdf = [np.NaN] * 3 #membentuk list kosong sebesar 3 
nilai_pdf[0] = f2(newData, sigma1, kelas1)
nilai_pdf[1] = f2(newData, sigma2, kelas2)
nilai_pdf[2]= f2(newData, sigma3, kelas3)

print (nilai_pdf)

[0.00061856019562038186, 0.0018125272764340384, 0.019357915884568862]


Dari hasil pdf diatas, dapat disimpulkan bahwa nilai terbesar didapat dari pdf3. Dengan kata lain, newData tergolong kelas 3 (label kelas = 2).

Sesimpel itu teman-teman. Tidak perlu ribet ribet memahami dan memaksakan model ANN pada model kalian. 
Algoritma diatas dapat dibungkus kedalam sebuah fungsi klasifikasi seperti berikut :


In [197]:
def klasifikasi_multivariate(data, g, kelas1, kelas2, kelas3, kamus_kelas):
    nilai_pdf = [np.NaN] * 3 #membentuk list kosong sebesar 3 
    nilai_pdf[0] = f2(data, G(g, kelas1), kelas1)
    nilai_pdf[1] = f2(data, G(g, kelas2), kelas2)
    nilai_pdf[2] = f2(data, G(g, kelas3), kelas3)
    minimum_index = nilai_pdf.index(min(nilai_pdf))
    return kamus_kelas[minimum_index-1]

Sehingga penerapan klasifikasi terhadap setiap data dapat lebih nyaman dilihat. COntoh kasus : 

In [201]:
kamus = [0,1,2]
g = 2.5

dataBaru = dataTest.loc[1]
label = klasifikasi_multivariate(dataBaru, g, kelas1, kelas2, kelas3, kamus)
print (label)


2


## B.1.c Output ke prediksi.txt

In [171]:
output = dataTest
output['label'] = dataTrain['label']

trace = go.Table(
    header=dict(values=output.columns,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[output.att1, output.att2, output.att3, output.label],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))


data = [trace] 
iplot(data, filename = 'Data Test')

In [20]:
output.to_csv('output.txt', header=True, index=False, sep='\t', mode='a')

## B.1.d Petunjuk pengujian (Readme.txt)
Sudah dijelaskan

## B.2 Akurasi (Terhadap data validasi)
Dapat diimplementasikan